[View in Colaboratory](https://colab.research.google.com/github/RayleighKim/M10_Intro_to_Classification/blob/master/Hackathon2_Churn_Prediction.ipynb)

# Hackathon2

## Churn Prediction!

### Your name:

#### 문제를 정의할 것
* 무엇을 할지
* 왜 하는 것인지
* 어떻게 활용할 것인지

#### 조건
* Data Wrangling & Featrue Engineering 할 것.
* 어떤 모델을 가져다가 사용해도 좋음
* Scikit Leanr을 사용하기를 권장함.

#### 목표 (둘중 하나를 만족할 것)
* 이탈을 설명하라.
* 성능!
    * (Optional) 돈으로 연결짓는다면? 다음과 같은 시나리오가 나오면 좋다!
        * 모델 성능은 유지된다고 가정
        * 지난 달의 신규 고객 1000명 대상
        * 실제 이탈 예정자들은 600명이 있다고 함.
        * 인당 10000원이 소모되는 캠페인 A가 있음.
        * 이탈예정자들은 A캠페인에 노출되면 평균적으로 15000원을 사용함
        * 이탈예정자들은 A캠페인에 노출되지 않으면 돈 안씀
        * 이탈안할 사람들은 A캠페인에 노출되든 안되든 그냥 와서 살만큼 삼 (A캠페인에 의한 매출 발생은 없다고 가정함)
        * 기존의 방식은 그냥 1000명 전부에게 A캠페인을 실행함.
        * 기존의 방식대로 하면 캠페인 비용 천만원, 캠페인으로 인한 매출 900만원.
        * 내 예측 모델을 이용해 예측된 이탈후보자들에게만 캠페인을 진행한다면?
---------------
Rayleigh Kim @ D:plus

In [0]:
'''
라이브러리들을 불러오자.
'''
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

## Data Loading & Preparation

실습에서 생략된 다음의 과정을 충실히 진행해야 한다!

Raw_data 에서, **어떤 과정**을 거치면 굉장히 잘 정돈된 Data를 얻게 된다.

**어떤 과정이란 다음과 같다.**
1. 원래 데이터는 어떻게 생겼는가?
2. 문제 정의
    * 이탈자는 어떻게 정의할 것인가.
    * 어떤 기간의 데이터로, 어떤 기간을 예측할 것인가.
3. **Data Wrangling & Feature Engineering**


다음의 데이터들을 구경해보자.

동시에, **(new주석)RetailChurnTemplate_FeatureEngg.xlsx** 파일을 열어서 감상하자.

RetailChurnTemplate_FeatureEngg.csv와 같은 형태의 파일로 만들어내는 것이 우리의 목표이다.

### 조건
1. 이탈에 대한 정의를 명확히 할 것. 아래는 예시
    * 최근 21일 이내 방문 없으면 이탈!
    * 그렇다면 최근 21사이 신규 고객 전부 제거해야 함!
    * 라벨링은 최근 21데이터로만 해야 함
    * Data Wrangling & Feature Engineering은 최근 21일 데이터를 떼고 해야 함!
2. Feature Engineering을 통해 Feature들을 만들어 낼 것!
    * 반드시 실습 파일과 똑같은 것들을 재현할 필요는 없음
    * 상상!


    

In [0]:
'''
고객 정보 데이터
'''
user_info = pd.read_csv('https://raw.githubusercontent.com/RayleighKim/Example_datasets/master/RetailChurn_UserInfoData.csv')
user_info.head()

In [0]:
'''
Transaction 데이터
'''

raw_data = pd.read_csv('https://raw.githubusercontent.com/RayleighKim/Example_datasets/master/RetailChurn_Transaction.csv')
raw_data.head()

## Feature Engineering!

### 위 두개의 데이터로 부터 아래와 같은 데이터를 만들어 내는 것이 목표다!

* Dummy variable은 아직 신경쓰지 말 것
* Group by, Join을 적극적으로 활용할 것!
* Pandas 실습 파일을 적극 참고할 것!

In [0]:
churn = pd.read_csv('https://raw.githubusercontent.com/RayleighKim/Example_datasets/master/RetailChurnTemplate_FeatureEngg.csv')
'''
아래의 코드는 row를 셔플하기 위해 필요하다.
'''
churn = churn.sample(frac = 1, random_state = 2018).reset_index(drop=True)

churn.head()

Your Code!

In [0]:
### Your Final Data

print(churn.columns)
churn.dtypes

## Dummy & Drop

In [0]:
# User는 따로 떼어 둬야 어떤 User인지 알 수 있다.
Users = churn[['UserId']]

## 아래의 코드를 수정하여 사용할 것!
## 버릴 컬럼들 꽤 많을수도 있음!
# Unknown 투성이인 Gender 와 UserType을 버리자. 
churn_dropped = churn.drop(['UserId', 'Gender', 'UserType'], axis=1)
churn_dropped.head()

In [0]:
# 어딘가에 있을지 모를 NA, NaN 등을 찾자. 
churn_dropped.isna().any()

더미를 이용하여 펼치자.

컴퓨터는 A, B, C, D라고 입력되어 있어봤자 그걸 그대로 인식할 수 없다!

In [0]:
churn_dropped.head()

In [0]:
pd.get_dummies(churn_dropped[['Age','Address']]).head()

In [0]:
churn_processed = pd.concat([churn_dropped, pd.get_dummies(churn_dropped[['Age','Address']])], axis=1).drop(['Age','Address'], axis=1)

churn_processed['label'] = 1
churn_processed.loc[churn_processed['Label']=='Nonchurner', 'label'] = 0

churn_ready = churn_processed.drop('Label', axis=1)

churn_ready.head()

## Train set, Test set

특별한 문제가 없는 한, 아래의 셀은 그대로 실행되어야 한다.

In [0]:
churn_processed.shape

In [0]:
X_train, X_test, Y_train, Y_test = churn_ready.loc[:7000].drop('label', axis=1), \
churn_ready.loc[7001:].drop('label', axis=1),\
churn_ready.loc[:7000, 'label'],\
churn_ready.loc[7001:, 'label']

In [0]:
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

## Modeling!

아래의 셀은 특별한 문제가 없다면 그냥 실행되어야 한다.

* 선택지1 : Feature Engineering만으로 승부를 본다. 위에서 Feature Engineering 해가며 아래 셀을 실행하여 성능을 관찰한다.
* 선택지2 : Scikit-Learn을 이용하여 여러 모델들을 사용해본다. 

In [0]:
clf = LogisticRegression()
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
Y_pred_train = clf.predict(X_train)

print('트레이닝 7001명 중 실제 churner의 비율 : ',Y_train.mean())
print('트레이닝 7001명에 대해서 예측된 churner의 비율 :', Y_pred_train.mean())
print('모델의 트레이닝셋위의 정확도는 : {:.2f}%'.format(clf.score(X_train, Y_train)*100))

print('--------------------------------------------')

print('테스트 n명 중 실제 churner의 비율 : ',Y_test.mean())
print('테스트 n명에 대해서 예측된 churner의 비율 :', Y_pred.mean())
print('모델의 테스트셋위의 정확도는 : {:.2f}%'.format(clf.score(X_test, Y_test)*100))


print('---------------------------------------------')
print(confusion_matrix(Y_test,Y_pred).T)


print('---------------------------------------------')
print('precision : True라고 예측한 것 중 진짜 True 비율')
print('recall : 진짜 True 중 True라고 예측된 것의 비율')
print(classification_report(Y_test, Y_pred, target_names=('Non-churner','churner')))
pd.DataFrame({'colnames':X_train.columns, 'coef':clf.coef_.reshape(-1,)})